### ANÁLISIS DE DATOS NO ESTRUCUTRADOS - PRÁCTICA TEXTO

### Comparativa de canciones de diferentes artistas

#### Álvaro Bartolomé Ucero

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

In [2]:
songs = pd.read_csv("data/hhgroups_merge_28_05.csv", header=0)
#songs.head()
songs = songs[songs.artista != 'Lata Mangeshkar']
songs.head(10)

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219
5,5,Khan,Lobo,"Sin álbum, es un vídeo suelto",Que nadie creía en leyends hasta que llegué yo...,2019,683
6,6,Dyo,Claridad [Productor desconocido],"Sin álbum, es un tema suelto","Me olvidé tomarme la pastilla otra vez,\nEstar...",2019,58
7,7,Sheldrack,Especial agradecimiento,Titiritero,Hace tiempo dejé de buscar el sentido de la vi...,2018,35
8,8,Arce,Huella (con Pitizion),Pedigrí,[Estribillo: Pitizon]\nNo te voy a negar que a...,2019,119
9,9,Arce,Boxer,Pedigrí,"Se preguntaron dónde está, me perdí como Wally...",2019,843


In [3]:
songs.count()

id         9325
artista    9325
cancion    9325
album      9325
letra      9325
anyo       9325
visitas    9325
dtype: int64

#### Echamos un ojo al número de canciones de cada artista, en orden descendente:

In [4]:
songs2 = songs.groupby('artista')['cancion'].nunique().sort_values(ascending=False)

songs2.head(10)

artista
Porta          157
SFDK           139
Nach           127
Toteking       101
El Chojin       91
ZPU             87
Rayden          76
Shotta          71
Juaninacka      71
Pablo Hasél     69
Name: cancion, dtype: int64

#### Porta y SFDK son los artistas con más canciones, crearé un nuevo DataFrame solo con sus canciones

In [5]:
songsPorta = songs.loc[songs['artista'] =='Porta']

In [6]:
songsPorta.head()

,id,artista,cancion,album,letra,anyo,visitas
10,10,Porta,Otra salida (con Uve Sad),Off,[Estribillo: Uve Sad]\nNunca pensé lo que serí...,2019,86
11,11,Porta,Rap&Roll,Off,"Nos dicen que recordemos los ideales, no al ho...",2019,95
12,12,Porta,Nunca volverá,Off,"El tiempo corre tan deprisa,\nQue sin darnos c...",2019,2868
192,27,Porta,Etiquetas,Reset,"Nos escondemos detrás de una imagen, de la apa...",2012,4053
237,17,Porta,Me dejo la piel,Inédito,"Bienvenido a Reset man, este es mi reinicio\np...",2008,4952


In [7]:
songsSFDK = songs.loc[songs['artista'] =='SFDK']

In [8]:
songsSFDK.head()

,id,artista,cancion,album,letra,anyo,visitas
160,50,SFDK,Duelo de vikingos (con Promoe),2001 Odisea en el lodo,[Promoe]\nEvery single time Promo and the Loop...,2003,222
161,51,SFDK,Esta canción va dedicada...,2001 Odisea en el lodo,[Zatu]\nPonme en on y se bienvenido al infinit...,2003,303
195,30,SFDK,"Los chicos del barrio (con El Límite, Dementor...",Lista de invitados,No para de sonar\nno vuelvas a llamar\nestoy c...,2011,8174
220,0,SFDK,Crazy love (con Zatu),Lista de invitados,"El hip hop era joven, ingenuo, confuso, a vece...",2011,4949
238,18,SFDK,Está to feo (con Rapsusklei),Lista de invitados,[Rapsus]: ¡Killo cabesa!\n[Zatu]: ¿Qué pasa co...,2011,24324


### Corpus global, corpus de Porta y corpus de SFDK

In [9]:
text_corpus = []
for song in songs['letra']:
    words = song.lower().split()
    text_corpus.append(words)



# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

9325


In [10]:
text_corpus_Porta = []
for song in songsPorta['letra']:
    words = song.lower().split()
    text_corpus_Porta.append(words)



# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vecPorta = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vecPorta.build_vocab(text_corpus_Porta)
print (len(text_corpus_Porta))

157


In [16]:
text_corpus_SFDK = []
for song in songsSFDK['letra']:
    words = song.lower().split()
    text_corpus_SFDK.append(words)



# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vecSFDK = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vecSFDK.build_vocab(text_corpus_SFDK)
print (len(text_corpus_SFDK))

140


In [17]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 24.816407442092896 seconds ---


In [18]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

In [19]:
import time
start_time = time.time()



songs2vecPorta.train(text_corpus_Porta, total_examples=songs2vecPorta.corpus_count, epochs=2)

if not os.path.exists("trained_Porta"):
    os.makedirs("trained_Porta")

songs2vecPorta.save(os.path.join("trained_Porta", "songs2vectorsPorta.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6552455425262451 seconds ---


In [20]:
songs2vecPorta = w2v.Word2Vec.load(os.path.join("trained_Porta", "songs2vectorsPorta.w2v"))

In [21]:
import time
start_time = time.time()



songs2vecSFDK.train(text_corpus_SFDK, total_examples=songs2vecSFDK.corpus_count, epochs=2)

if not os.path.exists("trained_SFDK"):
    os.makedirs("trained_SFDK")

songs2vecSFDK.save(os.path.join("trained_SFDK", "songs2vectorsSFDK.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.44880032539367676 seconds ---


In [22]:
songs2vecSFDK = w2v.Word2Vec.load(os.path.join("trained_SFDK", "songs2vectorsSFDK.w2v"))

### Comenzaremos explorando el modelo

#### Palabras similares en el csv de canciones de rap en español:

In [23]:
songs2vec.wv.most_similar("amor")

[('odio', 0.9041743278503418),
 ('amor,', 0.8949775695800781),
 ('sufrimiento', 0.8937614560127258),
 ('odio,', 0.8883864879608154),
 ('rencor,', 0.8855385780334473),
 ('cariño', 0.8756365776062012),
 ('silencio', 0.8732749223709106),
 ('rencor', 0.8725239038467407),
 ('afecto', 0.8707345128059387),
 ('engaño', 0.8706508874893188)]

In [24]:
songs2vec.wv.most_similar("mujer")

[('niña', 0.8917500972747803),
 ('hermana', 0.8758699297904968),
 ('chica', 0.8704174160957336),
 ('dama', 0.8701075315475464),
 ('zorra', 0.868372917175293),
 ('mujer,', 0.8668615221977234),
 ('diosa', 0.8623470067977905),
 ('amiga', 0.8617444038391113),
 ('hija,', 0.8615381717681885),
 ('bebé', 0.8614274263381958)]

#### Palabras similares dentro de las canciones de Porta

In [25]:
songs2vecPorta.wv.most_similar("amor")

[('principio', 0.9844077825546265),
 ('pasado', 0.9839773178100586),
 ('corazon', 0.9839168787002563),
 ('dolor', 0.9828122854232788),
 ('pasa', 0.9817530512809753),
 ('motivo', 0.9800866842269897),
 ('sueño', 0.979415774345398),
 ('jamas', 0.9775180816650391),
 ('destino', 0.9771683812141418),
 ('final', 0.977009654045105)]

In [27]:
songs2vecPorta.wv.most_similar("mujer")

[('despertar,', 0.9960626363754272),
 ('dijo', 0.9957952499389648),
 ('podría', 0.9957695603370667),
 ('dejare', 0.9954947233200073),
 ('recita', 0.9954348802566528),
 ('pueden,', 0.9953402280807495),
 ('sentido.', 0.9953399896621704),
 ('favor', 0.9952632784843445),
 ('contrato', 0.9952530264854431),
 ('resto,', 0.9952511787414551)]

#### Palabras similares dentro de las canciones de SFDK

In [28]:
songs2vecSFDK.wv.most_similar("amor")

[('camino', 0.99941486120224),
 ('sevilla', 0.9992952346801758),
 ('dolor', 0.9992252588272095),
 ('punto', 0.9992077350616455),
 ('tema', 0.9991751909255981),
 ('medio', 0.9991491436958313),
 ('sido', 0.9990565180778503),
 ('entra', 0.9990553259849548),
 ('to', 0.9990500211715698),
 ('flow', 0.9990317225456238)]

In [29]:
songs2vecSFDK.wv.most_similar("mujer")

[('tenemos', 0.9993696212768555),
 ('conozco', 0.9993577003479004),
 ('muchos', 0.9993538856506348),
 ('mcs', 0.9993177652359009),
 ('illo', 0.9992801547050476),
 ('estaba', 0.9992673397064209),
 ('primo', 0.9992453455924988),
 ('arriba', 0.9991984367370605),
 ('más,', 0.9991691708564758),
 ('puesto', 0.9991636872291565)]

Podemos sacar en claro una serie de puntos al comparar estas palabras similares extraidas:
    - Las palabras similares buscadas en el csv completo son más precisas lógicamente ya que es un vector mucho más grande.
    - Las palabras similares varían en función del artista, ya que cada uno tiene su estilo.

#### Palabras fuera de contexto en el csv de canciones de rap en español:

In [30]:
songs2vec.wv.doesnt_match("amor odio tristeza hombre".split())

C:\Anaconda\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'hombre'

In [31]:
songs2vec.most_similar(positive=['amor', 'odio'], negative=['hombre'])
#queen

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('odio,', 0.8217482566833496),
 ('rencor,', 0.75859534740448),
 ('sufrimiento', 0.7561668753623962),
 ('amo', 0.7548519372940063),
 ('amor,', 0.7476058602333069),
 ('desamor', 0.7308030128479004),
 ('rencor', 0.7289711236953735),
 ('dentro.', 0.7283638715744019),
 ('placer', 0.7278642058372498),
 ('aliento,', 0.7220728397369385)]

#### Palabras fuera de contexto dentro de las canciones de Porta

In [32]:
songs2vecPorta.wv.doesnt_match("amor odio tristeza hombre".split())

'tristeza'

In [33]:
songs2vecPorta.most_similar(positive=['amor', 'odio'], negative=['hombre'])
#queen

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('final', 0.96796715259552),
 ('pasado', 0.9676187038421631),
 ('cielo', 0.9642876386642456),
 ('principio', 0.9620102643966675),
 ('sueño', 0.9598523378372192),
 ('pasa', 0.9597901701927185),
 ('corazon', 0.9595530033111572),
 ('momento', 0.9566640257835388),
 ('fin', 0.9535763263702393),
 ('dolor', 0.9526416063308716)]

#### Palabras fuera de contexto dentro de las canciones de SFDK

In [34]:
songs2vecSFDK.wv.doesnt_match("amor odio tristeza hombre".split())

'tristeza'

In [35]:
songs2vecSFDK.most_similar(positive=['amor', 'odio'], negative=['hombre'])
#queen

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sido', 0.9990097284317017),
 ('directo', 0.998920738697052),
 ('punto', 0.9988793730735779),
 ('donde', 0.9988734722137451),
 ('¿qué', 0.9988614916801453),
 ('montano', 0.9988120794296265),
 ('aunque', 0.9987986087799072),
 ('ahi', 0.9987586736679077),
 ('dolor', 0.9987542033195496),
 ('visto', 0.9987497925758362)]

Al extraer palabras fuera de contexto sucede algo parecido que en el apartado anterior. En le caso de utilizar el 100% de las canciones del csv, el error disminuye enormemente. Sin embargo, al utilizar únicamente el 1.68% de las canciones (Porta) o el 1.5% de las canciones (SFDK) este error crece exponencialmente. De todas formas es curioso observar las relaciones que extrae en función del artista que se esté analizando. 

#### Distancia semántica entre palabras en el csv de canciones de rap en español:

In [36]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [37]:
nearest_similarity_cosmul("amor", "odio", "casa")

amor es a odio, lo que casa, es a casa


#### Distancia semántica entre palabras dentro de las canciones de Porta

In [38]:
def nearest_similarity_cosmul_Porta(start1, end1, end2):
    similarities = songs2vecPorta.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [39]:
nearest_similarity_cosmul_Porta("amor", "odio", "casa")

amor es a odio, lo que ritmo es a casa


#### Distancia semántica entre palabras dentro de las canciones de SFDK

In [40]:
def nearest_similarity_cosmul_SFDK(start1, end1, end2):
    similarities = songs2vecSFDK.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [41]:
nearest_similarity_cosmul_SFDK("amor", "odio", "casa")

amor es a odio, lo que esta es a casa


En esta tercera comparación tampoco hay demasiado que recalcar, que no hayamos comentado antes

#### Añadimos como columna el vector de canciones a los tres DataFrames creados y generaremos las coordenadas X e Y de cada canción

In [42]:
print(songs2vec['amor'])
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songs['song_vector'] = songs['letra'].apply(songVector)




C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher

[-0.17716727 -0.0679598  -0.5054434   0.15936491  0.4117319  -0.41066575
  0.28536308 -0.08642731 -0.04806018  0.19459946 -0.06519866 -0.2751113
  0.27664927  0.11007828 -0.2076547   0.07769069 -0.31141508  0.41684365
 -0.3063712   0.31585804 -1.3887528   0.3662654  -0.8579703   0.3590147
  0.40371302  0.6804785  -0.29042524  0.33118093 -0.6754868   0.38513654
 -0.07942837  0.03072833  0.2783746   0.08815054  0.6612455   0.41480544
  0.69556147  0.4189638  -0.24050446 -0.48476505 -0.20903468  0.13435666
 -0.7234201  -0.4217193  -0.3521341  -0.15960667 -0.06917219 -0.4131869
 -0.2517891  -0.90288335]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

...

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

In [43]:
print(songs2vecPorta['amor'])
def songVectorPorta(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vecPorta[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songsPorta['song_vector'] = songsPorta['letra'].apply(songVectorPorta)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher

[-0.31417853  0.24754901 -0.49703136  0.32550812  0.40646207  0.02059838
  0.41185507  0.9297036   0.36140957  0.11712327 -0.3487897   0.34288016
 -0.09739081 -0.18958163  0.20190114 -0.23947205 -0.33011982  0.3670328
 -0.19448717  0.12853621 -1.1266786   0.04994244 -0.44601727  0.03471268
  0.08423346  0.5245444  -0.01536533  0.45848238 -0.96141887  0.45323133
 -0.25553435 -0.04017585  0.12240324 -0.17049097  0.21862929  0.5532541
  0.47544843  0.2016185  -0.16024475 -0.32824203 -0.53274226  0.21677071
  0.07269828 -0.0543335  -0.02419258 -0.01243363 -0.22022463 -0.66575503
 -0.17335232 -0.14125027]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [44]:
print(songs2vecSFDK['amor'])
def songVectorSFDK(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vecSFDK[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songsSFDK['song_vector'] = songsSFDK['letra'].apply(songVectorSFDK)

[-0.35078534 -0.07646409 -0.48174646 -0.12038716  0.4522382  -0.4723499
  0.33677346  0.263136    0.10782183 -0.21051522  0.0509462   0.14187652
  0.48808932  0.05414216  0.5387784   0.16211568 -0.01902305  0.46678174
 -0.22051318  0.10332751 -1.194215    0.5011493  -0.7027749  -0.11797167
 -0.12620726  0.3116975  -0.5087847   0.48023504 -1.1955124   0.16997941
 -0.18986759  0.15845929  0.29287446 -0.41276723  0.15074974  0.5159633
  0.3518677   0.23322214 -0.667727   -0.2667647  -0.44561458 -0.31129158
  0.07162461  0.01762506 -0.10435454 -0.6847557  -0.4466743  -0.3447561
  0.19444093 -0.31694457]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Anaconda\lib\site-packages\ipykernel_launcher

In [74]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
7837,52,Asfalto,Líricas de largo y corto alcance,La lanza de los dioses,"Son clases prácticas herramientas básicas,\nmu...",2015,859,"[[0.038345393, 0.0008464969, -0.08039209, 0.07..."
8512,12,Zarcort,Bioshock infinite [Productor desconocido],Rap-play,[Estribillo] (x2)\nNo tengo ni idea de como he...,2013,4119,"[[0.06534308, 0.030395107, -0.046927955, 0.060..."
8988,48,Jotandjota,Ahí voy otra vez,Sincericidio,Tirao en el suelo del baño y no puedo levantar...,2012,1373,"[[0.045123037, 0.047765717, -0.061908744, 0.02..."
6685,0,Drake,Madonna,If you're reading this it's too late,"[Estribillo]\nBreathe through, ride through.\n...",2015,468,"[[0.04840272, -0.05853554, -0.08142776, -0.221..."
8010,5,Rayden,A mi yo de ayer,En alma y hueso,"Me pregunto si me oyes todavía,\nsi queda algo...",2014,59687,"[[0.06917446, 0.014409751, -0.080690846, 0.067..."
6464,54,Jona y Piezas,Sin rencor,Sin álbum,[Jona]\nAlgunas veces quisiera que vinieras co...,2012,3950,"[[0.050149783, 0.006778256, -0.06549176, 0.040..."
4398,53,Miranda,Diferente,Valknut,"Un fuego sin oxígeno siempre se va a morir,\nU...",2018,391,"[[0.04322365, 0.018476538, -0.09240214, 0.0588..."
6495,30,Trafik y Jompy,Luz en la oscuridad (con Nous Nizzy),Falta corazón,[Nous Nizzy]\nHay miradas que resaltan en el f...,2016,924,"[[0.055914745, 0.0028951685, -0.09885666, -0.0..."
5756,6,Recycled J y Selecta,Cadena,City pop,"Yo me compré un cordón pa' brillar,\nLo perdí ...",2019,1834,"[[0.015790187, 0.068062045, -0.052483466, 0.03..."
8232,7,Porta,Puro instinto (con Eddie MV),Algo ha cambiado,[Porta]\nSi encuentran esta grabación quizá es...,2014,2087,"[[0.045601305, 0.030439395, -0.06639614, 0.080..."


In [76]:
song_vectors_Porta = []
from sklearn.model_selection import train_test_split

trainPorta, testPorta = train_test_split(songsPorta, test_size = 0.9)


for song_vector_Porta in trainPorta['song_vector']:
    song_vectors_Porta.append(song_vector_Porta)

trainPorta.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
6549,29,Porta,Imperfecto,Equilibrio,"[Estribillo]\nI know I'm not perfect,\nI know ...",2016,3523,"[[-0.07755403, 0.05157057, -0.089679435, 0.061..."
9116,11,Porta,Sexualirap (con Dj Joaking),Reset,Dejo un aviso parental aqui hay contenido sexu...,2012,2042,"[[-0.087099195, 0.09670758, -0.082443066, 0.10..."
8824,49,Porta,Animales racionales,Reset,La racionalidad es esa capacidad humana que pe...,2012,3041,"[[-0.08444491, 0.09486104, -0.092535034, 0.111..."
8237,12,Porta,Bienvenidos (con H0lynaight),Algo ha cambiado,[Porta]\nBienvenidos sean a este circo\nEn est...,2014,2310,"[[-0.08667967, 0.091884576, -0.0963966, 0.1108..."
2979,9,Porta,Rumores. lo que se comenta por alli,No hay truco,Cuando uno adquiere un poco de fama la gente l...,2007,7407,"[[-0.079656795, 0.07934907, -0.09773619, 0.105..."
2812,7,Porta,No hay truco,No hay truco,[Soma]\nTodo lo que oigan en este trabajo musi...,2007,5105,"[[-0.08771104, 0.08317537, -0.09067131, 0.1058..."
2693,53,Porta,Una sociedad un tanto rara,En boca de tantos,Levanten las manos y amen al tonto de Hitler\n...,2008,5042,"[[-0.07823903, 0.09162152, -0.10251694, 0.1108..."
3029,4,Porta,Bienvenidos a mi show,No es cuestion de edades,"Yeah! Bienvenidos, esto no es cuestion de edad...",2006,3744,"[[-0.082138024, 0.106370755, -0.11681208, 0.11..."
3003,33,Porta,Mi realidad,No es cuestion de edades,No podemos demostrar que lo que vivimos es rea...,2006,4182,"[[-0.087886766, 0.106118694, -0.0998896, 0.118..."
3002,32,Porta,Voy a saco,No hay truco,"Soy de los MCs más queridos, pero de los más c...",2007,3504,"[[-0.079809204, 0.07645295, -0.09643447, 0.100..."


In [78]:
song_vectors_SFDK = []
from sklearn.model_selection import train_test_split

trainSFDK, testSFDK = train_test_split(songsSFDK, test_size = 0.9)


for song_vector_SFDK in trainSFDK['song_vector']:
    song_vectors_SFDK.append(song_vector_SFDK)

trainSFDK.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
5881,21,SFDK,El ingeniero,Siempre fuertes,Llego al microfono desconectalo\nY enciende el...,1999,1363,"[[-0.10541867, -0.0073568565, -0.16126536, -0...."
2663,23,SFDK,El Umbral (con A.D Green),Los veteranos,"Yo, SFDK, A.D.....es que si no podemos entrar ...",2007,3025,"[[-0.11061935, -0.014092918, -0.16652627, -0.0..."
7868,28,SFDK,La puñalá,Sin miedo a vivir,[Estribillo]\n¿Donde estará mi navaja?\nSé que...,2014,3046,"[[-0.1095807, -0.013645675, -0.1682284, -0.002..."
4326,36,SFDK,¿A donde van?,2001 Odisea en el lodo,Si en cada esquina que escupo salta un nuevo g...,2003,5563,"[[-0.10961708, -0.012204779, -0.16551419, -0.0..."
4118,48,SFDK,¿Yo que hago aquí? (con Jefe de la M),2005,"[Jefe De La M]\nAh... 2000 five es el año, el ...",2005,3075,"[[-0.10729582, -0.0123810135, -0.16507187, -0...."
3098,18,SFDK,3 Hombres y un destino (con Toteking),Desde los chiqueros,"Lamentable, Cartuja 93 bailando el tarabare ti...",2000,4851,"[[-0.10522272, -0.00835749, -0.15934765, -0.01..."
7017,2,SFDK,"Digan lo que quieran (con Little Pepe, Jefe de...",Lista de invitados,[Estribillo: Little Pepe] (x2)\nDeja que hable...,2011,1251,"[[-0.11333039, -0.015963832, -0.16829957, -0.0..."
4232,52,SFDK,Bailes de salón (con Juaninacka),2005,[Zatu]\nResulta que voy un dia camino de los j...,2005,4226,"[[-0.10785589, -0.013067828, -0.16750379, -0.0..."
4125,0,SFDK,Rap romántico,Desde los chiqueros,"Nunca voy ebrio mama, voy sobrio, perdona\nno ...",2000,29131,"[[-0.11004121, -0.011472046, -0.16317461, -0.0..."
9005,10,SFDK,El tiempo está aquí (con Spanish Fly),Lista de invitados,[Spanish Fly]\nAquí vuelve en angel de las ala...,2011,3195,"[[-0.10859164, -0.010189339, -0.16074225, -0.0..."


In [81]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 932 samples in 0.002s...
[t-SNE] Computed neighbors for 932 samples in 0.081s...
[t-SNE] Computed conditional probabilities for sample 932 / 932
[t-SNE] Mean sigma: 0.035318
[t-SNE] Computed conditional probabilities in 0.049s
[t-SNE] Iteration 50: error = 76.4731750, gradient norm = 0.3171903 (50 iterations in 0.448s)
[t-SNE] Iteration 100: error = 76.4197311, gradient norm = 0.3291267 (50 iterations in 0.462s)
[t-SNE] Iteration 150: error = 77.1941376, gradient norm = 0.3336252 (50 iterations in 0.461s)
[t-SNE] Iteration 200: error = 76.0842133, gradient norm = 0.3217860 (50 iterations in 0.474s)
[t-SNE] Iteration 250: error = 78.0504532, gradient norm = 0.2899915 (50 iterations in 0.422s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 78.050453
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895351438246

In [82]:
X_Porta = np.array(song_vectors_Porta).reshape((15, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d_Porta = tsne.fit_transform(X_Porta)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 14 nearest neighbors...
[t-SNE] Indexed 15 samples in 0.000s...
[t-SNE] Computed neighbors for 15 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 15 / 15
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] Computed conditional probabilities in 0.002s
[t-SNE] Iteration 50: error = 54.1237259, gradient norm = 0.0926645 (50 iterations in 0.006s)
[t-SNE] Iteration 100: error = 56.3986702, gradient norm = 0.2396644 (50 iterations in 0.005s)
[t-SNE] Iteration 150: error = 65.8044205, gradient norm = 0.8436421 (50 iterations in 0.006s)
[t-SNE] Iteration 200: error = 46.6639290, gradient norm = 0.0260053 (50 iterations in 0.006s)
[t-SNE] Iteration 250: error = 63.9630318, gradient norm = 0.5979643 (50 iterations in 0.006s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.963032
[t-SNE] KL divergence after 251 iterations: 179769313486231570814527423731704356798070567525844996598917476803157260780028538760589558632766878171540458

In [84]:
X_SFDK = np.array(song_vectors_SFDK).reshape((14, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d_SFDK = tsne.fit_transform(X_SFDK)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 13 nearest neighbors...
[t-SNE] Indexed 14 samples in 0.000s...
[t-SNE] Computed neighbors for 14 samples in 0.000s...
[t-SNE] Computed conditional probabilities for sample 14 / 14
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] Computed conditional probabilities in 0.001s
[t-SNE] Iteration 50: error = 53.2654533, gradient norm = 0.3540858 (50 iterations in 0.006s)
[t-SNE] Iteration 100: error = 47.5083466, gradient norm = 0.1879770 (50 iterations in 0.006s)
[t-SNE] Iteration 150: error = 53.3731499, gradient norm = 0.2044713 (50 iterations in 0.005s)
[t-SNE] Iteration 200: error = 56.9947128, gradient norm = 0.1372189 (50 iterations in 0.007s)
[t-SNE] Iteration 250: error = 59.3750458, gradient norm = 0.1421311 (50 iterations in 0.007s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 59.375046
[t-SNE] KL divergence after 251 iterations: 179769313486231570814527423731704356798070567525844996598917476803157260780028538760589558632766878171540458

In [85]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [86]:
dfPorta=pd.DataFrame(all_word_vectors_matrix_2d_Porta,columns=['X','Y'])

dfPorta.head(10)

trainPorta.head()

dfPorta.reset_index(drop=True, inplace=True)
trainPorta.reset_index(drop=True, inplace=True)

In [87]:
dfSFDK=pd.DataFrame(all_word_vectors_matrix_2d_SFDK,columns=['X','Y'])

dfSFDK.head(10)

trainSFDK.head()

dfSFDK.reset_index(drop=True, inplace=True)
trainSFDK.reset_index(drop=True, inplace=True)

In [88]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,52,Asfalto,Líricas de largo y corto alcance,La lanza de los dioses,"Son clases prácticas herramientas básicas,\nmu...",2015,859,"[[0.038345393, 0.0008464969, -0.08039209, 0.07...",0.118221,0.049154
1,12,Zarcort,Bioshock infinite [Productor desconocido],Rap-play,[Estribillo] (x2)\nNo tengo ni idea de como he...,2013,4119,"[[0.06534308, 0.030395107, -0.046927955, 0.060...",0.025208,0.201498
2,48,Jotandjota,Ahí voy otra vez,Sincericidio,Tirao en el suelo del baño y no puedo levantar...,2012,1373,"[[0.045123037, 0.047765717, -0.061908744, 0.02...",0.027824,0.062562
3,0,Drake,Madonna,If you're reading this it's too late,"[Estribillo]\nBreathe through, ride through.\n...",2015,468,"[[0.04840272, -0.05853554, -0.08142776, -0.221...",-0.541414,-0.954083
4,5,Rayden,A mi yo de ayer,En alma y hueso,"Me pregunto si me oyes todavía,\nsi queda algo...",2014,59687,"[[0.06917446, 0.014409751, -0.080690846, 0.067...",0.279919,0.206523


In [89]:
two_dimensional_songs_Porta = pd.concat([trainPorta, dfPorta], axis=1)

two_dimensional_songs_Porta.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,29,Porta,Imperfecto,Equilibrio,"[Estribillo]\nI know I'm not perfect,\nI know ...",2016,3523,"[[-0.07755403, 0.05157057, -0.089679435, 0.061...",-33.838753,-35.350578
1,11,Porta,Sexualirap (con Dj Joaking),Reset,Dejo un aviso parental aqui hay contenido sexu...,2012,2042,"[[-0.087099195, 0.09670758, -0.082443066, 0.10...",-54.849472,-23.346054
2,49,Porta,Animales racionales,Reset,La racionalidad es esa capacidad humana que pe...,2012,3041,"[[-0.08444491, 0.09486104, -0.092535034, 0.111...",0.355207,-117.656174
3,12,Porta,Bienvenidos (con H0lynaight),Algo ha cambiado,[Porta]\nBienvenidos sean a este circo\nEn est...,2014,2310,"[[-0.08667967, 0.091884576, -0.0963966, 0.1108...",-99.604195,68.522224
4,9,Porta,Rumores. lo que se comenta por alli,No hay truco,Cuando uno adquiere un poco de fama la gente l...,2007,7407,"[[-0.079656795, 0.07934907, -0.09773619, 0.105...",52.107033,-31.302769


In [90]:
two_dimensional_songs_SFDK = pd.concat([trainSFDK, dfSFDK], axis=1)

two_dimensional_songs_SFDK.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,21,SFDK,El ingeniero,Siempre fuertes,Llego al microfono desconectalo\nY enciende el...,1999,1363,"[[-0.10541867, -0.0073568565, -0.16126536, -0....",31.338079,84.630646
1,23,SFDK,El Umbral (con A.D Green),Los veteranos,"Yo, SFDK, A.D.....es que si no podemos entrar ...",2007,3025,"[[-0.11061935, -0.014092918, -0.16652627, -0.0...",44.125526,60.947327
2,28,SFDK,La puñalá,Sin miedo a vivir,[Estribillo]\n¿Donde estará mi navaja?\nSé que...,2014,3046,"[[-0.1095807, -0.013645675, -0.1682284, -0.002...",-15.078889,13.986301
3,36,SFDK,¿A donde van?,2001 Odisea en el lodo,Si en cada esquina que escupo salta un nuevo g...,2003,5563,"[[-0.10961708, -0.012204779, -0.16551419, -0.0...",41.067818,-71.104889
4,48,SFDK,¿Yo que hago aquí? (con Jefe de la M),2005,"[Jefe De La M]\nAh... 2000 five es el año, el ...",2005,3075,"[[-0.10729582, -0.0123810135, -0.16507187, -0....",-77.264450,-104.963943


Ahora ya tenemos creados los tres Dataframes: uno con todas las canciones, otro solo con las canciones de Porta y otro solo con las de SFDK, habiendo añadido las columnas de "song_vector" y las cooredandas X e Y de cada canción.

### Gráficos de los resultados

In [91]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

import plotly.graph_objects as go
import numpy as np

import plotly.express as px

In [92]:
fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'],
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

In [93]:
fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs_Porta['Y'],
    x = two_dimensional_songs_Porta['X'],
    text = two_dimensional_songs_Porta['artista']+ "_"+two_dimensional_songs_Porta['cancion'],
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

In [94]:
fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs_SFDK['Y'],
    x = two_dimensional_songs_SFDK['X'],
    text = two_dimensional_songs_SFDK['artista']+ "_"+two_dimensional_songs_SFDK['cancion'],
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

En estas tres figuras podemos ver las diferencias y similitudes entre las distintas canciones, en función de sus coordenadas X e Y. Se puede observar en la primera imagen (donde se están utilizando todas las canciones) que hay un elevado número de ellas muy parecidas entre sí.

Si nos centramos más en comparar las canciones de Porta entre sí y las canciones de SFDK entre sí, vemos lógicamente un menor número de canciones que en el caso previo y estas presentan ciertas similitudes.

En el caso de SFDK, se puede observar que hay un mayor número de canciones próximas al centro, lo que significa que utilizan un estilo y un vocabulario muy similar. Por otro lado, en el caso de Porta es algo diferente, ya que parece que sus canciones difieren bastante unas de otras. Esto ya depende del estilo musical de cada uno de los artistas.

In [96]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()